In [1]:
import numpy as np
import pandas as pd
import time

In [2]:
train=pd.read_csv('meinian_round1_train_20180408.csv',sep=',',encoding='gbk')
test=pd.read_csv('meinian_round1_test_b_20180505.csv',sep=',',encoding='gbk')
data_part1 = pd.read_csv("meinian_round1_data_part1_20180408.txt", sep="$")
data_part2 =pd.read_csv("meinian_round1_data_part2_20180408.txt", sep='$')

In [3]:
#test.shape[0]*(test.shape[1]-1)

In [4]:
#result=pd.read_csv("meinian4.csv")

In [5]:
#result.shape[0]*(result.shape[1]-1)

In [6]:
#(test["vid"]==result["vid"]).sum()*(result.shape[1]-1)

In [7]:
# data_part1和data_part2进行合并
part1_2 = pd.concat([data_part1,data_part2],axis=0)#{0/'index', 1/'columns'}, default 0
part1_2 = pd.DataFrame(part1_2).sort_values('vid').reset_index(drop=True)
#vid_set dataframe vid 病人集合 dataframe
vid_set=pd.concat([train['vid'],test['vid']],axis=0)
vid_set=pd.DataFrame(vid_set).sort_values('vid').reset_index(drop=True)
#并剔除掉与train、test不相关vid所在的数据行
part1_2=part1_2[part1_2['vid'].isin(vid_set['vid'])]

In [8]:
print(part1_2.shape)
vid_tabid_group = part1_2.groupby(['vid','table_id']).size().reset_index()
vid_tabid_group = part1_2.groupby(['vid','table_id']).size().reset_index()
print(vid_tabid_group.head())
print(vid_tabid_group.shape)

(6750883, 3)
                                vid table_id  0
0  000330ad1f424114719b7525f400660b     0101  1
1  000330ad1f424114719b7525f400660b     0102  3
2  000330ad1f424114719b7525f400660b     0113  1
3  000330ad1f424114719b7525f400660b     0114  1
4  000330ad1f424114719b7525f400660b     0115  1
(6515933, 3)


In [9]:
# 重塑index用来去重,区分重复部分和唯一部分
vid_tabid_group['new_index'] = vid_tabid_group['vid'] + '_' + vid_tabid_group['table_id']
#vid_tabid_group_dup 重复的vid_tabid 检查项
vid_tabid_group_dup = vid_tabid_group[vid_tabid_group[0]>1]['new_index']
print(vid_tabid_group_dup.head())

1      000330ad1f424114719b7525f400660b_0102
147    0003848ebd8d8163603760d53d975693_0101
148    0003848ebd8d8163603760d53d975693_0102
228    0003848ebd8d8163603760d53d975693_1308
299    000401cbf304d5a8bd862a81bacfa494_0101
Name: new_index, dtype: object


In [10]:
part1_2['new_index'] = part1_2['vid'] + '_' + part1_2['table_id']
dup_part = part1_2[part1_2['new_index'].isin(list(vid_tabid_group_dup))]
dup_part = dup_part.sort_values(['vid','table_id'])
unique_part = part1_2[~part1_2['new_index'].isin(list(vid_tabid_group_dup))]

In [11]:
# 重复数据的拼接操作
# 将vid_tabid 相同行 拼接
def merge_table(df):
    df['field_results'] = df['field_results'].astype(str)
    if df.shape[0] > 1:
        merge_df = " ".join(list(df['field_results']))
    else:
        merge_df = df['field_results'].values[0]
    return merge_df

In [12]:
part1_2_dup = dup_part.groupby(['vid','table_id']).apply(merge_table).reset_index()
part1_2_dup.rename(columns={0:'field_results'},inplace=True)
# 重复数据处理后 数据
part1_2_res = pd.concat([part1_2_dup,unique_part[['vid','table_id','field_results']]])

In [13]:
table_id_group=part1_2.groupby('table_id').size().sort_values(ascending=False)
table_id_group.to_csv('part_tabid_size.csv',encoding='utf-8')

In [14]:
# 行列转换 重新组织index vid 和columns  table_id
merge_part1_2 = part1_2_res.pivot(index='vid',values='field_results',columns='table_id')
print(merge_part1_2.shape)
merge_part1_2.to_csv('merge_part1_2.csv',encoding='utf-8')
#print(merge_part1_2.head())

(47731, 2736)


In [15]:
# 删除掉一些出现次数低，缺失比例大的字段，保留超过阈值的特征
def remain_feat(df,thresh=0.9):
    """
    删除掉一些出现次数低，缺失比例大的字段，保留超过阈值的特征,返回 需要保存的特征的list
    """
    exclude_feats = []
    print('----------移除数据缺失多的字段-----------')
    print('移除之前总的字段数量',len(df.columns))
    num_rows = df.shape[0]
    for c in df.columns:
        num_missing = df[c].isnull().sum()
        if num_missing == 0:
            continue
        missing_percent = num_missing / float(num_rows)
        if missing_percent > thresh:
            exclude_feats.append(c)
    print("移除缺失数据的字段数量: %s" % len(exclude_feats))
    # 保留超过阈值的特征
    feats = []
    for c in df.columns:
        if c not in exclude_feats:
            feats.append(c)
    print('剩余的字段数量',len(feats))
    return feats

In [16]:
feats=remain_feat(merge_part1_2,thresh=0.96)
merge_part1_2=merge_part1_2[feats]

----------移除数据缺失多的字段-----------
移除之前总的字段数量 2736
移除缺失数据的字段数量: 2334
剩余的字段数量 402


In [17]:
merge_part1_2.to_csv('merge_part1_2.csv')

In [18]:
merge_part1_2["vid"]=merge_part1_2.index
merge_part1_2.columns

Index(['004997', '0101', '0102', '0104', '0105', '0106', '0107', '0108',
       '0109', '0113',
       ...
       '979027', 'A201', 'A202', 'A301', 'A302', 'A601', 'A701', 'A703',
       'A705', 'vid'],
      dtype='object', name='table_id', length=403)

In [19]:
# 划分训练集 测试机 并且 同5个指标拼接
train_of_part=merge_part1_2[merge_part1_2['vid'].isin(train['vid'])]
test_of_part=merge_part1_2[merge_part1_2['vid'].isin(test['vid'])]

train=pd.merge(train,train_of_part,on='vid')
test=pd.merge(test,test_of_part,on='vid')

In [20]:
train.columns

Index(['vid', '收缩压', '舒张压', '血清甘油三酯', '血清高密度脂蛋白', '血清低密度脂蛋白', '004997', '0101',
       '0102', '0104',
       ...
       '979026', '979027', 'A201', 'A202', 'A301', 'A302', 'A601', 'A701',
       'A703', 'A705'],
      dtype='object', length=408)

In [29]:
# 清洗训练集中的五个指标
def clean_label(x):
    x=str(x)
    if '+' in x:#16.04++
        i=x.index('+')
        x=x[0:i]
    if '>' in x:#> 11.00
        i=x.index('>')
        x=x[i+1:]
    if len(x.split(sep='.'))>2:#2.2.8
        i=x.rindex('.')
        x=x[0:i]+x[i+1:]
    if '未做' in x or '未查' in x or '弃查' in x:
        x=np.nan
    if str(x).isdigit()==False and len(str(x))>4:
        x=x[0:4]
        pass
    x=float(x)
    if x>900:
        x=np.nan
    return x

In [30]:
targets=['收缩压', '舒张压', '血清甘油三酯', '血清高密度脂蛋白', '血清低密度脂蛋白']
for c in targets:
    train[c]=train[c].apply(clean_label)
    train[c]=train[c].astype('float64')    
    pass

In [31]:
train[targets].describe()

,收缩压,舒张压,血清甘油三酯,血清高密度脂蛋白,血清低密度脂蛋白
count,38192.000000,38190.000000,38199.000000,38199.000000,38199.000000
mean,126.049618,76.998115,1.616563,1.406683,2.769720
std,19.275625,12.580344,1.344201,0.341184,0.852187
min,0.000000,0.000000,0.100000,0.150000,-1.200000
25%,112.000000,68.000000,0.880000,1.160000,2.180000
50%,124.000000,76.000000,1.270000,1.350000,2.690000
75%,137.000000,85.000000,1.910000,1.600000,3.260000
max,252.000000,148.000000,28.800000,4.780000,11.500000


In [33]:
#保存train_set和test_set
train.to_csv('train_set.csv',index=False,encoding='utf-8')
test.to_csv('test_set.csv',index=False,encoding='utf-8')